# Capstone Project: Applied Data Science Capstone

## Recommender engine for a supplier

### Start by importing needed libraries

In [2]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.19.0-py_0       conda-forge

The following packages will be UPDATED:

   

### Preparing dataset 

In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from ipywidgets import *
from IPython.display import display

In [8]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

In [10]:
My_table = soup.find('table',{'class':'wikitable sortable'})

print(My_table.tr.text)


Postcode
Borough
Neighbourhood



In [11]:
headers="Postcode,Borough,Neighbourhood"

In [12]:
table1=""
for tr in My_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
print(table1)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Not assigned
M8A,Not assigned,Not assigned
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M

In [13]:
file=open("toronto.csv","wb")
file.write(bytes(table1,encoding="ascii",errors="ignore"))

8738

In [23]:
import pandas as pd
df = pd.read_csv('toronto.csv',header=None)
df.columns=["Postcode","Borough","Neighbourhood"]

In [24]:
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[ df['Borough'] =='Not assigned'].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [25]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [26]:
result = df.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)

df_new=result.reset_index()
df_new.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [29]:
#getting clean version of the dataframe
df_new=result.reset_index()
df_new.head()
print('The shape of the Dataframe is',df_new.shape)

The shape of the Dataframe is (103, 3)


In [30]:
#read geospatial data
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
#assign headers
df_lon_lat.columns=['Postcode','Latitude','Longitude']

df_lon_lat.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
df_toronto = pd.merge(df_new,
                 df_lon_lat[['Postcode','Latitude', 'Longitude']],
                 on='Postcode')
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Toronto map with Postcodes

In [34]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [45]:
# Extracting neighborhoods within "Etobicoke"
Etobicoke_data = df_toronto[df_toronto['Borough'] == 'Etobicoke']
Etobicoke_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
11,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
17,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
70,M9P,Etobicoke,Westmount,43.696319,-79.532242
77,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724


## Draw map of Etobicoke and its neighborhoods


In [50]:

address_eto = 'Etobicoke, Toronto'
latitude_eto = 43.6205
longitude_eto = -79.5132
print('The geograpical coordinate of "Etobicoke" are: {}, {}.'.format(latitude_eto, longitude_eto))

map_Etobicoke = folium.Map(location=[latitude_eto, longitude_eto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Etobicoke_data['Latitude'], Etobicoke_data['Longitude'], Etobicoke_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Etobicoke)  
    
map_Etobicoke

The geograpical coordinate of "Etobicoke" are: 43.6205, -79.5132.


In [51]:
CLIENT_ID = 'YPBIDRZL5OM31BRJXMQWPTOVLPFV3ADXUEYUEEFGYUAYQB4L' # your Foursquare ID
CLIENT_SECRET = 'QVCKWK4HB01JQBSSGBVJMCWVQIZY0N0XNH5PEIXZQTSBRZNF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Client ID is ', CLIENT_ID )
print('Client Secret is ', CLIENT_SECRET )

Client ID is  YPBIDRZL5OM31BRJXMQWPTOVLPFV3ADXUEYUEEFGYUAYQB4L
Client Secret is  QVCKWK4HB01JQBSSGBVJMCWVQIZY0N0XNH5PEIXZQTSBRZNF


In [52]:

def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

## Crawling neighborhoods and venues in Etobicoke

In [53]:
print('Crawling different neighborhoods inside "Etobicoke"')
Etobicoke_foursquare_dataset = foursquare_crawler(list(Etobicoke_data['Postcode']),
                                                   list(Etobicoke_data['Neighbourhood']),
                                                   list(Etobicoke_data['Latitude']),
                                                   list(Etobicoke_data['Longitude']),)

Crawling different neighborhoods inside "Etobicoke"
1.
Data is Obtained, for the Postal Code M9A (and Neighborhoods Islington Avenue) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M9B (and Neighborhoods Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M9C (and Neighborhoods Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M9P (and Neighborhoods Westmount) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M9R (and Neighborhoods Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M8V (and Neighborhoods Humber Bay Shores, Mimico South, New Toronto) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M9V (and Neighborhoods Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown) SUCCESSFULLY.
8.
Data 

## Saving final results from Foursquare, in order to save resources without the need to reconnect every time to Foursquare

In [54]:
import pickle
with open("Etobicoke_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Etobicoke_foursquare_dataset, fp)
print('Foursquare results file is Saved to Computer.')

Foursquare results file is Saved to Computer.


In [56]:
with open("Etobicoke_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    Etobicoke_foursquare_dataset = pickle.load(fp)
Etobicoke_foursquare_dataset

[{'Crawling_result': [{'reasons': {'count': 0,
     'items': [{'reasonName': 'globalInteractionReason',
       'summary': 'This spot is popular',
       'type': 'general'}]},
    'referralId': 'e-0-4bfd53764cf820a13849ecf4-0',
    'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
        'suffix': '.png'},
       'id': '4bf58dd8d48988d16d941735',
       'name': 'Café',
       'pluralName': 'Cafés',
       'primary': True,
       'shortName': 'Café'}],
     'id': '4bfd53764cf820a13849ecf4',
     'location': {'address': '1500 Islington Ave',
      'cc': 'CA',
      'city': 'Toronto',
      'country': 'Canada',
      'crossStreet': 'at Rathburn Rd',
      'distance': 600,
      'formattedAddress': ['1500 Islington Ave (at Rathburn Rd)',
       'Toronto ON M9A 3L8',
       'Canada'],
      'labeledLatLngs': [{'label': 'display',
        'lat': 43.662460906352436,
        'lng': -79.53205381416235}],
      'lat': 43.662460906352436,
      'lng

In [58]:
# Now we create a function to connect to the Foursquare file we saved. its Objective is to extract venues for every neighborhood inside the database

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [59]:
Etobicoke_venues = get_venue_dataset(Etobicoke_foursquare_dataset)

Number of Venuse in Coordination "M9A" Posal Code and "Islington Avenue" Negihborhood(s) is:
13
Number of Venuse in Coordination "M9B" Posal Code and "Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park" Negihborhood(s) is:
17
Number of Venuse in Coordination "M9C" Posal Code and "Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe" Negihborhood(s) is:
13
Number of Venuse in Coordination "M9P" Posal Code and "Westmount" Negihborhood(s) is:
13
Number of Venuse in Coordination "M9R" Posal Code and "Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips" Negihborhood(s) is:
14
Number of Venuse in Coordination "M8V" Posal Code and "Humber Bay Shores, Mimico South, New Toronto" Negihborhood(s) is:
20
Number of Venuse in Coordination "M9V" Posal Code and "Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown" Negihborhood(s) is:
15
Number of Venuse in Coordination "M8W" Posal Code and "Alder

In [60]:
Etobicoke_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M9A,Islington Avenue,43.667856,-79.532242,Java Joe's Village Cafe,This spot is popular,Café,600
1,M9A,Islington Avenue,43.667856,-79.532242,St Georges Golf and Country Club,This spot is popular,Golf Course,827
2,M9A,Islington Avenue,43.667856,-79.532242,TD Canada Trust,This spot is popular,Bank,579
3,M9A,Islington Avenue,43.667856,-79.532242,Shoppers Drug Mart,This spot is popular,Pharmacy,521
4,M9A,Islington Avenue,43.667856,-79.532242,Cobs Bread,This spot is popular,Bakery,1000


In [62]:
# Save processed data to file
Etobicoke_venues.to_csv('Etobicoke_venues.csv')

In [63]:
Etobicoke_venues = pd.read_csv('Etobicoke_venues.csv')

In [64]:
neigh_list = list(Etobicoke_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Etobicoke:')
print(len(neigh_list))
print('List of Neighborhoods inside Etobicoke:')
neigh_list

Number of Neighborhoods inside Etobicoke:
12
List of Neighborhoods inside Etobicoke:


['Islington Avenue',
 'Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park',
 'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe',
 'Westmount',
 'Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips',
 'Humber Bay Shores, Mimico South, New Toronto',
 'Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown',
 'Alderwood, Long Branch',
 'Northwest',
 'The Kingsway, Montgomery Road, Old Mill North',
 "Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",
 'Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor']

In [65]:
neigh_venue_summary = Etobicoke_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",15,15,15,15,15,15,15
"Alderwood, Long Branch",26,26,26,26,26,26,26
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",13,13,13,13,13,13,13
"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park",17,17,17,17,17,17,17
"Humber Bay Shores, Mimico South, New Toronto",20,20,20,20,20,20,20


In [67]:
print('There are {} uniques categories.'.format(len(Etobicoke_venues['Venue Category'].unique())))

print('List of different categories:')
list(Etobicoke_venues['Venue Category'].unique())

There are 90 uniques categories.
List of different categories:


['Café',
 'Golf Course',
 'Bank',
 'Pharmacy',
 'Bakery',
 'Grocery Store',
 'Shopping Mall',
 'Playground',
 'Park',
 'Skating Rink',
 'Convenience Store',
 'Fish & Chips Shop',
 'American Restaurant',
 'Pizza Place',
 'Gym',
 'Theater',
 'Mexican Restaurant',
 'Hotel',
 'Clothing Store',
 'Liquor Store',
 'Coffee Shop',
 'Beer Store',
 'College Rec Center',
 'Farmers Market',
 'IT Services',
 'Chinese Restaurant',
 'Sandwich Place',
 'Supermarket',
 'Intersection',
 'Middle Eastern Restaurant',
 'Discount Store',
 'Ice Cream Shop',
 'Golf Driving Range',
 'Breakfast Spot',
 'Bus Line',
 'Mobile Phone Shop',
 'Supplement Shop',
 'Dessert Shop',
 'Italian Restaurant',
 'Restaurant',
 'Indian Restaurant',
 'Pub',
 'Fried Chicken Joint',
 'Fast Food Restaurant',
 'Seafood Restaurant',
 'Hardware Store',
 'Moroccan Restaurant',
 'Pool',
 'Athletics & Sports',
 'Gas Station',
 'Donut Shop',
 'Trail',
 'Garden Center',
 'Dog Run',
 'Sushi Restaurant',
 'French Restaurant',
 'Tapas Restauran

In [68]:
# one hot encoding
Etobicoke_onehot = pd.get_dummies(data = Etobicoke_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
Etobicoke_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Adult Boutique,American Restaurant,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Beer Store,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Bus Line,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Comfort Food Restaurant,Convenience Store,Cupcake Shop,Deli / Bodega,Dessert Shop,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Fried Chicken Joint,Garden Center,Gas Station,Gastropub,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Laundry Service,Liquor Store,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Pub,Restaurant,River,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Social Club,Spa,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Wings Joint,Yoga Studio
0,0,M9A,Islington Avenue,43.667856,-79.532242,Java Joe's Village Cafe,This spot is popular,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M9A,Islington Avenue,43.667856,-79.532242,St Georges Golf and Country Club,This spot is popular,827,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M9A,Islington Avenue,43.667856,-79.532242,TD Canada Trust,This spot is popular,579,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M9A,Islington Avenue,43.667856,-79.532242,Shoppers Drug Mart,This spot is popular,521,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M9A,Islington Avenue,43.667856,-79.532242,Cobs Bread,This spot is popular,1000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [84]:
# This list is created manually 
important_list_of_features = [
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'American Restaurant',
 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Breakfast Spot',
 'Burger Joint',
 'Chinese Restaurant',
 'Fast Food Restaurant',
 'Fried Chicken Joint',
 'Greek Restaurant',
 'Grocery Store',
 'Indian Restaurant',
 'Italian Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Pizza Place',
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Sushi Restaurant',
 'Thai Restaurant',
 'Wings Joint']

In [85]:
Etobicoke_onehot = Etobicoke_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


Etobicoke_onehot.head()

,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Chinese Restaurant,Fast Food Restaurant,Fried Chicken Joint,Greek Restaurant,Grocery Store,Indian Restaurant,Italian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Thai Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",0,0,0,0,0,0,0,1,1,0,3,0,0,0,0,0,3,0,1,0,0,0,0
"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,1,0,0,0,0
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park",1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0,0
"Humber Bay Shores, Mimico South, New Toronto",1,0,0,1,0,0,0,1,1,0,1,1,1,0,1,0,1,1,1,1,0,0,0


## Assumption: the above mentioned places use mostly same row ingredients 

In [87]:
feat_name_list = list(Etobicoke_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
Etobicoke_onehot['Total Restaurants'] = Etobicoke_onehot[restaurant_list].sum(axis = 1)
Etobicoke_onehot = Etobicoke_onehot.drop(columns = restaurant_list)


feat_name_list = list(Etobicoke_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
Etobicoke_onehot['Total Joints'] = Etobicoke_onehot[joint_list].sum(axis = 1)
Etobicoke_onehot = Etobicoke_onehot.drop(columns = joint_list)

In [88]:
Etobicoke_onehot

,Bakery,Breakfast Spot,Grocery Store,Pizza Place,Sandwich Place
Neighborhood,,,,,
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",0,0,3,3,1
"Alderwood, Long Branch",0,0,2,2,1
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",0,0,1,1,0
"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park",0,0,1,2,0
"Humber Bay Shores, Mimico South, New Toronto",1,0,1,1,1
"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",0,0,0,0,0
Islington Avenue,1,0,1,0,0
"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",0,0,0,1,1
"Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor",2,0,1,0,2


## Clustering

In [90]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(Etobicoke_onehot)

In [92]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = Etobicoke_onehot.columns
means_df.index = ['A1','A2','A3','A4','A5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Grocery Store,Pizza Place,Sandwich Place,Total Sum
A2,0.000000,0.0,2.50,2.500000,1.000000,6.0
A4,2.000000,0.0,1.00,0.000000,2.000000,5.0
A3,0.500000,1.5,0.00,2.000000,0.500000,4.5
A5,0.333333,0.0,1.00,1.333333,0.333333,3.0
A1,0.250000,0.0,0.25,0.250000,0.250000,1.0


### Outcome: best group is A2

In [98]:
neigh_summary = pd.DataFrame([Etobicoke_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",2
1,"Alderwood, Long Branch",2
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",5
3,"Cloverdale, Islington, Martin Grove, Princess ...",5
4,"Humber Bay Shores, Mimico South, New Toronto",5
5,"Humber Bay, King's Mill Park, Kingsway Park So...",1
6,Islington Avenue,1
7,"Kingsview Village, Martin Grove Gardens, Richv...",1
8,"Kingsway Park South West, Mimico NW, The Queen...",4
9,Northwest,1


## Extracting best neighborhood ==> A2


In [106]:
neigh_summary[neigh_summary['Group'] == 2]

,Neighborhood,Group
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",2
1,"Alderwood, Long Branch",2


In [100]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 5]['Neighborhood'])[0]
Etobicoke_venues[Etobicoke_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Neighborhood': 'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe',
 'Neighborhood Latitude': 43.643515200000003,
 'Neighborhood Longitude': -79.577200799999986,
 'Postal Code': 'M9C'}

## Extracting best neighborhood ==> A4

In [107]:
neigh_summary[neigh_summary['Group'] == 4]

,Neighborhood,Group
8,"Kingsway Park South West, Mimico NW, The Queen...",4


## Extracting best neighborhood ==> A3

In [108]:
neigh_summary[neigh_summary['Group'] == 3]

,Neighborhood,Group
10,"The Kingsway, Montgomery Road, Old Mill North",3
11,Westmount,3


In [105]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 4]['Neighborhood'])[0]
Etobicoke_venues[Etobicoke_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Neighborhood': 'Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor',
 'Neighborhood Latitude': 43.628840799999999,
 'Neighborhood Longitude': -79.520999400000022,
 'Postal Code': 'M8Z'}